In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

from tqdm import trange

import model, utils, inference

from importlib import reload
reload(model)
reload(utils)
reload(inference)

<module 'inference' from '/storage/coda1/p-awu36/0/cli726/vis/pohawkes/inference.py'>

In [6]:
decay = 5
dt = 0.05
window_size = 5
n_neurons = 5
n_vis_neurons = 3
basis = utils.exp_basis(decay, window_size, dt*window_size)
T = 5
n_time_bins = int(T/dt)
n_samples = 60 # same number of sequences for train and test
n_samples_train = 40
n_trials = 10

In [17]:
df = pd.DataFrame(index=np.arange(n_trials), columns=['gen_model', 'spikes_list_train', 'convolved_spikes_list_train', 'firing_rates_list_train', 'spikes_list_test', 'convolved_spikes_list_test', 'firing_rates_list_test'])

In [18]:
for trial in trange(n_trials):
    torch.manual_seed(trial)
    
    gen_model = model.POGLM(n_neurons, n_vis_neurons, basis)
    with torch.no_grad():
        gen_model.linear.weight.data = torch.randn((n_neurons, n_neurons)) * 0.1
        gen_model.bias = torch.randn(n_neurons) * 0.1
    
    df.at[trial, 'gen_model'] = gen_model.state_dict()
    
    spikes_list, convolved_spikes_list, firing_rates_list = gen_model.sample(n_time_bins, n_samples)
    df.at[trial, 'spikes_list_train'] = spikes_list[:n_samples_train]
    df.at[trial, 'convolved_spikes_list_train'] = convolved_spikes_list[:n_samples_train]
    df.at[trial, 'firing_rates_list_train'] = firing_rates_list[:n_samples_train]
    df.at[trial, 'spikes_list_test'] = spikes_list[n_samples_train:]
    df.at[trial, 'convolved_spikes_list_test'] = convolved_spikes_list[n_samples_train:]
    df.at[trial, 'firing_rates_list_test'] = firing_rates_list[n_samples_train:]

100%|██████████| 10/10 [00:00<00:00, 41.05it/s]


In [19]:
df.to_pickle('data.pkl')